### Data Loading

In [1]:
import torch
import torch.nn as nn
import yaml
import glob
import scipy.io as scio
import random
import numpy as np
import os
# from mmfi_mmwave import make_dataset, make_dataloader
from mmfi_new import make_dataset, make_dataloader #for Radar_Fused
from tqdm import tqdm
from evaluate import error
from mmwave_point_transformer import PointTransformerReg

In [2]:
dataset_root = '/media/xinyan/My Passport/FYP/Data'
# xian zai shi yong de shi Radar_Fused
with open('config.yaml', 'r') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

train_dataset, val_dataset = make_dataset(dataset_root, config)

# rng_generator = torch.manual_seed(config['init_rand_seed'])
# train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'])
# val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'])

# # TODO: Code for training or validation
# sample = train_dataset[0]
# print(sample)

In [25]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    
    input_modalities = list(batch[0].keys())[-1]
    ## get kpts
    kpts = []
    [kpts.append(np.array(t['output'])) for t in batch]
    kpts = torch.FloatTensor(np.array(kpts))
    ## get sequence lengths
    lengths = torch.tensor([t[input_modalities].shape[0] for t in batch ])
    ## padd
    batch = [torch.Tensor(t[input_modalities]) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    batch = batch.permute(1,0,2)
    mask = (batch != 0)

    return batch, kpts, lengths, mask

In [3]:
rng_generator = torch.manual_seed(config['init_rand_seed'])
train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
# train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'])
# val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'])

In [4]:
for i,data in enumerate(train_loader):
    inputs, labels, length, mask = data
    print(inputs.shape)
    print(labels.shape)
    if i == 3:
        break

torch.Size([32, 65, 5])
torch.Size([32, 17, 3])
torch.Size([32, 61, 5])
torch.Size([32, 17, 3])
torch.Size([32, 53, 5])
torch.Size([32, 17, 3])
torch.Size([32, 56, 5])
torch.Size([32, 17, 3])


### creating a config

In [14]:
from configparser import ConfigParser

#Get the configparser object
config_object = ConfigParser()

#Assume we need 2 sections in the config file, let's call them USERINFO and SERVERCONFIG
config_object["Lidar"] = {
    'num_point': 1024, 
    'nblocks': 5, 
    'nneighbor': 16, 
    'num_class': 17*3, 
    'input_dim': 3
}
config_object["mmwave"] = {
    'nblocks': 5, 
    'nneighbor': 16, 
    'num_class': 17*3, 
    'input_dim': 5
}

#Write the above sections to config.ini file
with open('model_config.yaml', 'w') as conf:
    config_object.write(conf)

### model

In [6]:
model = PointTransformerReg(
    input_dim = 5,
    nblocks = 5,
    n_p = 17
)

In [7]:
# print(model)
x = torch.rand(16, 72, 5)
y = model(x)
print(y.shape)

torch.Size([16, 17, 3])


### training

In [7]:
def test(model, tensor_loader, criterion1, criterion2, device):
    model.eval()
    test_mpjpe = 0
    test_pampjpe = 0
    test_mse = 0
    for data in tqdm(tensor_loader):
        inputs, labels, _, _ = data
        inputs = inputs.to(device)
        labels.to(device)
        labels = labels.type(torch.FloatTensor)
        outputs = model(inputs)
        outputs = outputs.type(torch.FloatTensor)
        outputs.to(device)
        test_mse += criterion1(outputs,labels).item() * inputs.size(0)

        outputs = outputs.detach().numpy()
        labels = labels.detach().numpy()
        
        mpjpe, pampjpe = criterion2(outputs,labels)
        test_mpjpe += mpjpe.item() * inputs.size(0)
        test_pampjpe += pampjpe.item() * inputs.size(0)
    test_mpjpe = test_mpjpe/len(tensor_loader.dataset)
    test_pampjpe = test_pampjpe/len(tensor_loader.dataset)
    test_mse = test_mse/len(tensor_loader.dataset)
    print("mse: {:.8f}, mpjpe: {:.8f}, pampjpe: {:.8f}".format(float(test_mse), float(test_mpjpe),float(test_pampjpe)))
    return test_mpjpe

In [9]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('./pre-train_weights/mmwave_p1_random.pt'))
model.to(device)
test(
    model=model,
    tensor_loader= val_loader,
    criterion1 = train_criterion,
    criterion2 = test_criterion,
    device=device
        )

100%|██████████| 859/859 [00:58<00:00, 14.64it/s]

mse: 0.00249224, mpjpe: 0.07249401, pampjpe: 0.04329943


0.07249401098909201

In [8]:
def train(model, train_loader, test_loader, num_epochs, learning_rate, train_criterion, test_criterion, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
    # optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[20,40],gamma=0.1)
    parameter_dir = './pre-train_weights/mmwave_p2_cross_subject(2).pt'
    best_test_mpjpe = 100
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        epoch_accuracy = 0
        for data in tqdm(train_loader):
            inputs, labels, length, _ = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.type(torch.FloatTensor)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.to(device)
            outputs = outputs.type(torch.FloatTensor)
            loss = train_criterion(outputs,labels)
            loss.backward()
            # print(length)
            # print("loss is ", loss.item())
            optimizer.step()
            
            epoch_loss += loss.item() * inputs.size(0)
            # print("epoch loss is ", epoch_loss)
        epoch_loss = epoch_loss/len(train_loader.dataset)
        print('Epoch: {}, Loss: {:.8f}'.format(epoch, epoch_loss))
        if (epoch+1) % 5 == 0:
            test_mpjpe = test(
                model=model,
                tensor_loader=test_loader,
                criterion1 = train_criterion,
                criterion2 = test_criterion,
                device= device
            )
            if test_mpjpe <= best_test_mpjpe:
                print(f"best test mpjpe is:{test_mpjpe}")
                best_test_mpjpe = test_mpjpe
                torch.save(model.state_dict(), parameter_dir)
        # scheduler.step()
    return

In [9]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.load_state_dict(torch.load('./pre-train_weights/lidar_p1_random.pt'))
model.to(device)
train(
    model=model,
    train_loader= train_loader,
    test_loader= val_loader,    
    num_epochs= 50,
    learning_rate=1e-3,
    train_criterion = train_criterion,
    test_criterion = test_criterion,
    device=device 
        )

100%|██████████| 774/774 [01:28<00:00,  8.77it/s]


Epoch: 0, Loss: 0.04090454


100%|██████████| 774/774 [01:25<00:00,  9.02it/s]


Epoch: 1, Loss: 0.00949164


100%|██████████| 774/774 [01:25<00:00,  9.09it/s]


...

Epoch: 48, Loss: 0.00291014


100%|██████████| 774/774 [01:20<00:00,  9.65it/s]


Epoch: 49, Loss: 0.00283410


100%|██████████| 203/203 [00:10<00:00, 19.20it/s]

mse: 0.00911957, mpjpe: 0.14253297, pampjpe: 0.05833507
best test mpjpe is:0.14253297212979718


In [11]:
for i in train_dataset:
    print(i)
    print(i['input_lidar'].shape)
    break

{'modality': ['lidar'], 'scene': 'E01', 'subject': 'S02', 'action': 'A02', 'idx': 0, 'output': tensor([[ 0.0508,  0.6183,  2.8169],
        [ 0.0190,  0.6544,  2.8812],
        [ 0.0781,  0.6600,  2.8859],
        [-0.0256,  0.6572,  3.0327],
        [ 0.1287,  0.6689,  3.0403],
        [-0.0733,  0.4787,  3.0638],
        [ 0.1982,  0.5117,  3.2712],
        [-0.3137,  0.3794,  3.2431],
        [ 0.3910,  0.3150,  2.8655],
        [-0.2314,  0.4967,  3.5834],
        [ 0.2114,  0.3039,  2.2418],
        [-0.0221,  0.0094,  2.9235],
        [ 0.1611,  0.0140,  3.2860],
        [-0.0179, -0.3970,  2.9418],
        [ 0.1691, -0.4529,  3.3647],
        [-0.0062, -0.8078,  3.2431],
        [ 0.1699, -0.8288,  3.3098]]), 'input_lidar': array([[ 2.90777755,  0.07454614,  0.5344317 ],
       [ 2.91734099,  0.03896374,  0.53596509],
       [ 2.91168737,  0.00316235,  0.53494281],
       ...,
       [ 2.01373601, -0.24735571, -1.1397754 ],
       [ 2.00200009, -0.27087188, -1.13472259],
       

In [4]:
frame = '/media/xinyan/My Passport/FYP/Data/E01/S02/A02/lidar/frame001.bin'
_, mod = os.path.split(frame)
print(mod)

frame001.bin


In [5]:
import os

root_dir='/media/xinyan/My Passport/FYP/Data'
data_list = glob.glob(root_dir+'/*/*/*/radar')
data_list.sort()
for old_folder in data_list:
    new_folder = old_folder.replace('radar', 'mmwave')
    os.rename(old_folder, new_folder)

In [17]:
import shutil
root_dir='/home/xinyan/Desktop/FYP/Data'
data_list = glob.glob(root_dir+'/*/*/*/radar/*.bin')
data_list.sort()
for old_file in data_list:
    new_file = old_file.replace('/home/xinyan/Desktop/FYP/Data', '/media/xinyan/My Passport/FYP/Data')
    frame = '/' + new_file.split('/')[-1]
    new_folder = new_file.replace(frame, '')
    if not os.path.exists(new_folder):
        os.mkdir(new_folder)
    shutil.move(old_file, new_file)
# print(data_list)